In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
pesu1=pd.read_json('pesu1.json')
pesu2=pd.read_json('pesu2.json')
#pesu3=pd.read_json('pesu3_tweets.json')
#pesu4=pd.read_json('pesu4_tweets.json')
#pesu5=pd.read_json('pesu5_tweets.json')
#pesu6=pd.read_json('pesu6_tweets.json')

In [3]:
tweets_df = pd.concat([pesu1,pesu2], axis=0)
tweets_df = tweets_df.loc[:, ['text' , 'username' ,'tweet_id', 'timestamp' ,'hashtags' , 'screen_name','retweets','likes']]
tweets_df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes
0,"Bangalore youth, Bhaarat Chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0,0
1,Join our EdTech conference 11th/12th July at W...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0,0
2,This looks interesting: Join our EdTech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1,1
3,"Bangalore youth, Bhaarat Chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0,0
4,Join our EdTech conference 11th/12th July at W...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0,0
5,This looks interesting: Join our EdTech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1,1
6,Just posted a photo @ PES University https://w...,Manjunath Huded,1152189050071199744,2019-07-19 12:10:50,[],Manjunath_huded,0,0
7,I’m am genuinely thinking to boycott EA now. T...,Muhammad Abdullah,1152172736673996800,2019-07-19 11:06:01,[],MuhammadAYousuf,0,1
8,Be sure to come to PES University on the 20th!...,Varun Gupta,1151911316874907648,2019-07-18 17:47:14,[],varungupta3009,0,2
9,Drop in to PES University on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"[dscpesu, google, googlecloudpic]",dscpesu,0,3


In [4]:
tweets_df.reset_index(drop=True, inplace=True)

In [5]:
tweets_df.tail()

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes
982,Great Learning launches Masters’ program in Da...,Indiaeducationdiary,1234757042088415232,2020-03-03 08:26:34,[],indiaedudiary,0,0
983,"Doreswamy, who’s the chancellor of PES Univers...",Pratyush Singh,1234532240597504000,2020-03-02 17:33:17,[AmitShahIstifaDo],Pratyush_369,0,0
984,24 hours challenge #dotSlash 2.0 #hackathon at...,Shankar Ganesh PJ,1234389558638174208,2020-03-02 08:06:19,"[dotSlash, hackathon, university, organizers, ...",spillaiy,0,0
985,5th Mar 2020: IEEE PES Singapore Chapter is te...,IEEE PES Singapore Chapter,1234341646092783616,2020-03-02 04:55:56,[],PES_Singapore,2,3
986,PES University among Top 50 Management Institu...,PES University,1234114210000326656,2020-03-01 13:52:11,[],PESUniversity,1,3


In [6]:
tweets_df = tweets_df[~tweets_df.index.duplicated()]
uni_tweets = tweets_df.loc[tweets_df['screen_name'].isin(['PESUniversity','dscpesu','pesu_io'])]

In [7]:
uni_tweets['uni_handle'] = 1
uni_tweets

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle
9,Drop in to PES University on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"[dscpesu, google, googlecloudpic]",dscpesu,0,3,1
11,Our first cloud fest at PES UNIVERSITY! Launch...,DSC PESU,1151479175691132928,2019-07-17 13:10:03,"[dsc, google, cloud, gdg, googledevelopers, pe...",dscpesu,1,12,1
16,Drop in to PES University on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"[dscpesu, google, googlecloudpic]",dscpesu,0,3,1
18,Our first cloud fest at PES UNIVERSITY! Launch...,DSC PESU,1151479175691132928,2019-07-17 13:10:03,"[dsc, google, cloud, gdg, googledevelopers, pe...",dscpesu,1,12,1
46,Infini 2019 held\n\nFifty one Engineering coll...,PES University,1191740391546068992,2019-11-05 15:33:45,[],PESUniversity,0,5,1
47,Social awareness program on addiction with a p...,PES University,1190268835028320256,2019-11-01 14:06:19,[],PESUniversity,0,6,1
54,Infini 2019 held\n\nFifty one Engineering coll...,PES University,1191740391546068992,2019-11-05 15:33:45,[],PESUniversity,0,5,1
55,Social awareness program on addiction with a p...,PES University,1190268835028320256,2019-11-01 14:06:19,[],PESUniversity,0,6,1
63,"Finance Domain of MBA, PES University, success...",PES University,1172156293810778112,2019-09-12 14:33:32,[],PESUniversity,0,2,1
75,"Finance Domain of MBA, PES University, success...",PES University,1172156293810778112,2019-09-12 14:33:32,[],PESUniversity,0,2,1


In [8]:
tweets_df = tweets_df[~tweets_df.index.duplicated()]
tweets_df['uni_handle'] = uni_tweets['uni_handle']
pesu = tweets_df.combine_first(tweets_df)
pesu

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle
0,"Bangalore youth, Bhaarat Chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0,0,NaN
1,Join our EdTech conference 11th/12th July at W...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0,0,NaN
2,This looks interesting: Join our EdTech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1,1,NaN
3,"Bangalore youth, Bhaarat Chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0,0,NaN
4,Join our EdTech conference 11th/12th July at W...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0,0,NaN
5,This looks interesting: Join our EdTech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1,1,NaN
6,Just posted a photo @ PES University https://w...,Manjunath Huded,1152189050071199744,2019-07-19 12:10:50,[],Manjunath_huded,0,0,NaN
7,I’m am genuinely thinking to boycott EA now. T...,Muhammad Abdullah,1152172736673996800,2019-07-19 11:06:01,[],MuhammadAYousuf,0,1,NaN
8,Be sure to come to PES University on the 20th!...,Varun Gupta,1151911316874907648,2019-07-18 17:47:14,[],varungupta3009,0,2,NaN
9,Drop in to PES University on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"[dscpesu, google, googlecloudpic]",dscpesu,0,3,1.0


In [9]:
pesu[pesu['uni_handle'] == 1]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle
9,Drop in to PES University on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"[dscpesu, google, googlecloudpic]",dscpesu,0,3,1.0
11,Our first cloud fest at PES UNIVERSITY! Launch...,DSC PESU,1151479175691132928,2019-07-17 13:10:03,"[dsc, google, cloud, gdg, googledevelopers, pe...",dscpesu,1,12,1.0
16,Drop in to PES University on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"[dscpesu, google, googlecloudpic]",dscpesu,0,3,1.0
18,Our first cloud fest at PES UNIVERSITY! Launch...,DSC PESU,1151479175691132928,2019-07-17 13:10:03,"[dsc, google, cloud, gdg, googledevelopers, pe...",dscpesu,1,12,1.0
46,Infini 2019 held\n\nFifty one Engineering coll...,PES University,1191740391546068992,2019-11-05 15:33:45,[],PESUniversity,0,5,1.0
47,Social awareness program on addiction with a p...,PES University,1190268835028320256,2019-11-01 14:06:19,[],PESUniversity,0,6,1.0
54,Infini 2019 held\n\nFifty one Engineering coll...,PES University,1191740391546068992,2019-11-05 15:33:45,[],PESUniversity,0,5,1.0
55,Social awareness program on addiction with a p...,PES University,1190268835028320256,2019-11-01 14:06:19,[],PESUniversity,0,6,1.0
63,"Finance Domain of MBA, PES University, success...",PES University,1172156293810778112,2019-09-12 14:33:32,[],PESUniversity,0,2,1.0
75,"Finance Domain of MBA, PES University, success...",PES University,1172156293810778112,2019-09-12 14:33:32,[],PESUniversity,0,2,1.0


In [10]:
pesu['uni_handle'] = pesu['uni_handle'].replace(np.nan, 0)
pesu[pesu['uni_handle'] == 1]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle
9,Drop in to PES University on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"[dscpesu, google, googlecloudpic]",dscpesu,0,3,1.0
11,Our first cloud fest at PES UNIVERSITY! Launch...,DSC PESU,1151479175691132928,2019-07-17 13:10:03,"[dsc, google, cloud, gdg, googledevelopers, pe...",dscpesu,1,12,1.0
16,Drop in to PES University on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"[dscpesu, google, googlecloudpic]",dscpesu,0,3,1.0
18,Our first cloud fest at PES UNIVERSITY! Launch...,DSC PESU,1151479175691132928,2019-07-17 13:10:03,"[dsc, google, cloud, gdg, googledevelopers, pe...",dscpesu,1,12,1.0
46,Infini 2019 held\n\nFifty one Engineering coll...,PES University,1191740391546068992,2019-11-05 15:33:45,[],PESUniversity,0,5,1.0
47,Social awareness program on addiction with a p...,PES University,1190268835028320256,2019-11-01 14:06:19,[],PESUniversity,0,6,1.0
54,Infini 2019 held\n\nFifty one Engineering coll...,PES University,1191740391546068992,2019-11-05 15:33:45,[],PESUniversity,0,5,1.0
55,Social awareness program on addiction with a p...,PES University,1190268835028320256,2019-11-01 14:06:19,[],PESUniversity,0,6,1.0
63,"Finance Domain of MBA, PES University, success...",PES University,1172156293810778112,2019-09-12 14:33:32,[],PESUniversity,0,2,1.0
75,"Finance Domain of MBA, PES University, success...",PES University,1172156293810778112,2019-09-12 14:33:32,[],PESUniversity,0,2,1.0


In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
pesu['compound'] = [analyzer.polarity_scores(v)['compound'] for v in pesu['text']]

In [12]:
def sentiment(compound):
    res = ""
    if(compound >= 0.05):
        res="Positive"
    elif(compound <= - 0.05):
        res="Negative"
    else:
        res="Neutral"
    return res

pesu['Sentiment'] = pesu['compound'].apply(lambda x: sentiment(x))

In [13]:
pesu

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment
0,"Bangalore youth, Bhaarat Chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0,0,0.0,0.7096,Positive
1,Join our EdTech conference 11th/12th July at W...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0,0,0.0,0.2960,Positive
2,This looks interesting: Join our EdTech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1,1,0.0,0.5994,Positive
3,"Bangalore youth, Bhaarat Chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0,0,0.0,0.7096,Positive
4,Join our EdTech conference 11th/12th July at W...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0,0,0.0,0.2960,Positive
5,This looks interesting: Join our EdTech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1,1,0.0,0.5994,Positive
6,Just posted a photo @ PES University https://w...,Manjunath Huded,1152189050071199744,2019-07-19 12:10:50,[],Manjunath_huded,0,0,0.0,0.0000,Neutral
7,I’m am genuinely thinking to boycott EA now. T...,Muhammad Abdullah,1152172736673996800,2019-07-19 11:06:01,[],MuhammadAYousuf,0,1,0.0,0.1525,Positive
8,Be sure to come to PES University on the 20th!...,Varun Gupta,1151911316874907648,2019-07-18 17:47:14,[],varungupta3009,0,2,0.0,0.3802,Positive
9,Drop in to PES University on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"[dscpesu, google, googlecloudpic]",dscpesu,0,3,1.0,-0.2732,Negative


In [14]:
pesu['score'] = pd.cut(pesu['compound'], bins=5, labels=[1, 2, 3, 4, 5])
pesu

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,"Bangalore youth, Bhaarat Chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0,0,0.0,0.7096,Positive,5
1,Join our EdTech conference 11th/12th July at W...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0,0,0.0,0.2960,Positive,4
2,This looks interesting: Join our EdTech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1,1,0.0,0.5994,Positive,5
3,"Bangalore youth, Bhaarat Chetty, a final year ...",Balaji,1135790891866689536,2019-06-04 06:10:25,[],krbalaji68,0,0,0.0,0.7096,Positive,5
4,Join our EdTech conference 11th/12th July at W...,BPS_PES,1134844303447117824,2019-06-01 15:29:00,[],BPS_PES,0,0,0.0,0.2960,Positive,4
5,This looks interesting: Join our EdTech confer...,BPS_PES,1134844217665216512,2019-06-01 15:28:40,[],BPS_PES,1,1,0.0,0.5994,Positive,5
6,Just posted a photo @ PES University https://w...,Manjunath Huded,1152189050071199744,2019-07-19 12:10:50,[],Manjunath_huded,0,0,0.0,0.0000,Neutral,3
7,I’m am genuinely thinking to boycott EA now. T...,Muhammad Abdullah,1152172736673996800,2019-07-19 11:06:01,[],MuhammadAYousuf,0,1,0.0,0.1525,Positive,3
8,Be sure to come to PES University on the 20th!...,Varun Gupta,1151911316874907648,2019-07-18 17:47:14,[],varungupta3009,0,2,0.0,0.3802,Positive,4
9,Drop in to PES University on the 20th to meet ...,DSC PESU,1151910678363381760,2019-07-18 17:44:41,"[dscpesu, google, googlecloudpic]",dscpesu,0,3,1.0,-0.2732,Negative,2


In [16]:
pesu.to_csv('../../Datasets/final_pesu_withlikes.csv')